In [1]:
# Imports
import numpy as np
import pandas as pd

In [ ]:
# Finding the wavelength of laser beam

# CONSTANTS and INPUTS:
d = np.array([...]) # input in mm bro
N = np.array([...])                
delta = ...
least_count = ... # Least count of micrometer (mm)
uncertainty_N = ... # Counting uncertainty, idk what it would be

def calc_lambda(d, N, delta): 
    '''
    d is displacement of mirror for N fringes (mm)
    N is number of fringes counted
    delta is calibration constant of micrometer, idk wtf it is
    '''
    
    Lambda = ((2 * d) * delta) / N 
    return Lambda
                                       
WL = calc_lambda(d, N, delta)

mean_wl = np.mean(WL)
std_dev = np.std(WL, ddof=1) 
standard_error = std_dev / np.sqrt(len(WL)) # Standard Error of the Mean

avg_d = np.mean(d)
avg_N = np.mean(N)

frac_error_d = least_count / avg_d
frac_error_N = uncertainty_N / avg_N

total_frac_error = np.sqrt(frac_error_d**2 + frac_error_N**2)
uncertainty = mean_wl * total_frac_error

print(f"Calculated Wavelengths in mm: \n{WL}")
print(f"Mean Wavelength: {mean_wl:.7f} mm")
print(f"Std Dev: {std_dev:.7f} mm")
print(f"SE: {standard_error:.7f} mm")
print(f"Uncertainty: {uncertainty:.7f} mm")

In [ ]:
# Thickness of glass - take 3 measurements:
Thick = np.array([...]) # in mm again bro

print(f"Mean Thickness: {np.mean(Thick):.3f}")
print(f"Std Dev Thickness: {np.std(Thick, ddof = 1):.3f}")

In [ ]:
# Finding the Refractive Index of Glass Material

# CONSTANTS and INPUTS
T = np.mean(Thick) # Thickness of glass in mm again bro
N_glass = np.array([...]) # Fringes shifted
theta_glass = np.array([...]) # Angle in degrees
                                    
least_count_T = ... # Micrometer least count (mm)
least_count_theta = ... # Angle least count (degrees)
uncertainty_N = ...

def calc_ri(T, N, theta_deg, WL): 
    '''
    T is thickness of glass slide (mm)
    N is number of fringes shifted
    theta_deg is rotation angle in degrees
    WL is  wavelength of laser (mm)
    '''
    
    theta_rad = np.radians(theta_deg)
    
    X = 1 - np.cos(theta_rad)
    
    numerator = ((2 * T) - (N * WL)) * X
    denominator = (2 * T * X) - (N * WL)
    
    return numerator / denominator

def calc_uncert(T, N, theta_deg, WL, u_T, u_N, u_theta_deg):
    
    theta_rad = np.radians(theta_deg)
    u_theta_rad = np.radians(u_theta_deg)
    
    X = 1 - np.cos(theta_rad)
    D = (2 * T * X) - (N * WL) 
    
    deriv_N = (2 * T * WL * X * np.cos(theta_rad)) / (D**2)
    err_N = deriv_N * u_N
    
    deriv_T = (-2 * N * WL * X * np.cos(theta_rad)) / (D**2)
    err_T = deriv_T * u_T
    
    deriv_Theta = (-N * WL * (2 * T - N * WL) * np.sin(theta_rad)) / (D**2)
    err_Theta = deriv_Theta * u_theta_rad
    
    return np.sqrt(err_N**2 + err_T**2 + err_Theta**2)

RI_M = calc_ri(T, N_glass, theta_glass, mean_wl)

mean_ri = np.mean(RI_M)
std_dev_ri = np.std(RI_M, ddof=1)
standard_error_ri = std_dev_ri / np.sqrt(len(RI_M))

avg_N_glass = np.mean(N_glass)
avg_theta_glass = np.mean(theta_glass)

uncertainty_ri = calc_uncert(
    T, avg_N_glass, avg_theta_glass, mean_wl, 
    least_count_T, uncertainty_N, least_count_theta
)

print(f"Calculated Refractive Indices: \n{RI_M}")
print(f"Mean Refractive Index: {mean_ri:.4f}")
print(f"Std Dev: {std_dev_ri:.4f}")
print(f"SE: {standard_error_ri:.4f}")
print(f"Uncertainty: {uncertainty_ri:.4f}")

In [ ]:
# Finding the Refractive Index of Air

# CONSTANTS and INPUTS
D = ... # Length of Air Cell (mm)
P_atm = ... # Atmospheric Pressure (mmHg)
WL_m = mean_wl

N3 = np.array([...]) # Fringes counted
P_f = np.array([...]) # Final Pressure

uncertainty_N = ...
uncertainty_D = ... # Ruler (mm)
uncertainty_dP = ... # Pressure gauge (mmHg)

def calc_ri_air(N, WL, P_atm, D, dP): 
    '''
    N is number of fringes shifted
    WL is wavelength of laser (mm)
    P_atm is atmospheric pressure (mmHg)
    D is Length of the Air Cell (mm)
    dP is change in pressure (mmHg)
    '''

    part = (N * WL * P_atm) / (2 * D * dP)
    return 1 + part

def calc_uncert(N, P_atm, D, dP, n_mean, u_N, u_D, u_dP):
    
    frac_N = u_N / N
    frac_D = u_D / D
    frac_dP = u_dP / dP
    
    total_frac_error = np.sqrt(frac_N**2 + frac_D**2 + frac_dP**2)
    
    abs_error = (n_mean - 1) * total_frac_error
    return abs_error

dP = P_f - P_atm # Change in Pressure

RI_AIR = calc_ri_air(N3, WL_m, P_atm, D, dP)

mean_ri_air = np.mean(RI_AIR)
std_dev_air = np.std(RI_AIR, ddof=1)
standard_error_air = std_dev_air / np.sqrt(len(RI_AIR)) # Standard Error of the Mean

avg_N3 = np.mean(N3)
avg_dP = np.mean(dP)

uncertainty_air = calc_uncert(
    avg_N3, P_atm, D, avg_dP, mean_ri_air,
    uncertainty_N, uncertainty_D, uncertainty_dP
)

print(f"Calculated Refractive Indices: \n{RI_AIR}")
print(f"Mean Refractive Index: {mean_ri_air:.7f}")
print(f"Std Dev: {std_dev_air:.7f}")
print(f"SE: {standard_error_air:.7f}")
print(f"Uncertainty: {uncertainty_air:.7f}")